In [14]:
import heapq

class PuzzleState:
    def __init__(self, board, empty_pos, moves=0, parent=None):
        self.board = board  # Current board state
        self.empty_pos = empty_pos  # Position of the empty tile (0)
        self.moves = moves  # Number of moves from the start state
        self.parent = parent  # Parent state to trace the solution path

    def __lt__(self, other):
        # Compare the f(n) values for A* (f(n) = g(n) + h(n))
        return (self.moves + self.manhattan_distance()) < (other.moves + other.manhattan_distance())

    def manhattan_distance(self):
        """Compute the total Manhattan distance of all tiles."""
        distance = 0
        size = int(len(self.board) ** 0.5)  # Calculate the puzzle size (n x n)
        for i, tile in enumerate(self.board):
            if tile != 0:
                target_row, target_col = divmod(tile - 1, size)
                current_row, current_col = divmod(i, size)
                distance += abs(current_row - target_row) + abs(current_col - target_col)
        return distance

    def get_neighbors(self):
        """Generate all possible neighbors (states) from the current state."""
        neighbors = []
        size = int(len(self.board) ** 0.5)
        empty_row, empty_col = divmod(self.empty_pos, size)

        # Move the empty tile in 4 possible directions: up, down, left, right
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for dr, dc in directions:
            new_row, new_col = empty_row + dr, empty_col + dc
            if 0 <= new_row < size and 0 <= new_col < size:
                new_empty_pos = new_row * size + new_col
                new_board = list(self.board)
                # Swap the empty tile with the adjacent tile
                new_board[self.empty_pos], new_board[new_empty_pos] = new_board[new_empty_pos], new_board[self.empty_pos]
                neighbors.append(PuzzleState(new_board, new_empty_pos, self.moves + 1, self))

        return neighbors

class NPuzzleSolver:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state

    def a_star_search(self):
        """Solve the puzzle using A* algorithm."""
        open_list = []
        closed_list = set()
        heapq.heappush(open_list, self.initial_state)

        while open_list:
            current_state = heapq.heappop(open_list)

            # Check if the goal state is reached
            if current_state.board == self.goal_state:
                return self.reconstruct_path(current_state)

            closed_list.add(tuple(current_state.board))

            for neighbor in current_state.get_neighbors():
                if tuple(neighbor.board) not in closed_list:
                    heapq.heappush(open_list, neighbor)

        return None

    def reconstruct_path(self, goal_state):
        """Reconstruct the path from the goal state to the initial state."""
        path = []
        current_state = goal_state
        while current_state:
            path.append(current_state.board)
            current_state = current_state.parent
        return path[::-1]  # Reverse the path to get the solution from start to goal

def print_board(board):
    """Print the board in a readable format."""
    size = int(len(board) ** 0.5)
    for i in range(size):
        print(' '.join(str(board[i * size + j]) if board[i * size + j] != 0 else ' ' for j in range(size)))
    print()

def main():
    # Define the initial and goal states
    n = int(input("Enter the size of the puzzle (n for an n x n puzzle): "))
    size = n * n
    initial_state = list(map(int, input(f"Enter the initial state as {size} space-separated numbers (0 for the empty space): ").split()))
    goal_state = [i for i in range(1, size)] + [0]

    # Create the initial state object
    empty_pos = initial_state.index(0)
    initial_puzzle = PuzzleState(initial_state, empty_pos)

    # Solve the puzzle using A* algorithm
    solver = NPuzzleSolver(initial_puzzle, goal_state)
    solution = solver.a_star_search()

    if solution:
        print("Solution found:")
        for step in solution:
            print_board(step)
    else:
        print("No solution found.")

if __name__ == "__main__":
    main()


Enter the size of the puzzle (n for an n x n puzzle): 3
Enter the initial state as 9 space-separated numbers (0 for the empty space): 0 7 8
No solution found.
